In [ ]:
# Lets import all required libraries
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI


In [20]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

openai = OpenAI()
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

API key looks good so far


In [4]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are a Technical expert that specializes in Computer programming \
languages like Python, Javascript and Java while also having expertise in Data science and AL-ML and \
provides detailed explanations of code and concepts. \
Respond in markdown."

In [6]:

def messages_for(question):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ]

In [9]:
def answer_openai(question):
    response = openai.chat.completions.create(
        model = MODEL_GPT,
        messages = messages_for(question)
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [14]:
def answer_openai_stream(question):
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [17]:
def answer_ollama_stream(question):
    stream = ollama_via_openai.chat.completions.create(
        model=MODEL_LLAMA,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [18]:
question = "What is the difference between a list and a tuple in Python?"
answer_openai_stream(question)

In Python, both lists and tuples are used to store collections of items. However, they have several important differences that affect how they can be used in a program. Below are the key distinctions between lists and tuples:

### 1. **Mutability**

- **List**: Lists are mutable, which means you can modify them after their creation. You can add, remove, or change items in a list.
  
  python
  my_list = [1, 2, 3]
  my_list[1] = 5  # Changing the second element
  my_list.append(4)  # Adding a new element
  print(my_list)  # Output: [1, 5, 3, 4]
  

- **Tuple**: Tuples are immutable, meaning that once they are created, you cannot change their contents. You cannot add or remove items from a tuple.
  
  python
  my_tuple = (1, 2, 3)
  # my_tuple[1] = 5  # This will raise a TypeError
  

### 2. **Syntax**

- **List**: Lists are created using square brackets `[]`.

  python
  my_list = [1, 2, 3]
  

- **Tuple**: Tuples are created using parentheses `()`.

  python
  my_tuple = (1, 2, 3)
  

### 3. **Performance**

- **List**: Lists have more overhead associated with their dynamic resizing, and they are generally slower than tuples.

- **Tuple**: Tuples are more memory efficient and have a slight performance advantage since they are immutable. This can lead to faster access times.

### 4. **Use Cases**

- **List**: Commonly used when you need a collection of items that may change throughout the lifecycle of the program, such as a list of user inputs or a collection of items that needs to grow or shrink.

- **Tuple**: Typically used for fixed collections of items, such as coordinates (x, y), RGB values, or when you need to return multiple values from a function.

### 5. **Functions and Methods**

- **List**: Lists come with many built-in methods such as `append()`, `remove()`, `extend()`, and `pop()` that enable modification.

  python
  my_list.remove(2)  # Removes the first occurrence of 2
  

- **Tuple**: Tuples have fewer built-in methods. You can use methods like `count()` and `index()`, but you cannot modify the tuple.

  python
  print(my_tuple.count(1))  # Outputs the number of occurrences of 1
  

### 6. **Hashability**

- **List**: Lists are not hashable, which means they cannot be used as keys in dictionaries or elements in sets.

- **Tuple**: Tuples are hashable (as long as they contain only hashable elements) and can therefore be used as keys in dictionaries or stored in sets.

### Example

Here's a brief comparison in code:

python
# List example
list_example = [1, 2, 3]
list_example.append(4)  # Modify list
print(list_example)  # Output: [1, 2, 3, 4]

# Tuple example
tuple_example = (1, 2, 3)
# tuple_example.append(4)  # This would raise an AttributeError
print(tuple_example)  # Output: (1, 2, 3)


### Summary

In summary, lists are mutable, ordered collections enclosed in square brackets, while tuples are immutable, ordered collections enclosed in parentheses. The choice between using a list or a tuple depends on whether you need a collection that can change or one that should remain constant.

In [21]:
question = """Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}"""
answer_ollama_stream(question)

**Code Explanation**
===============

The provided code snippet utilizes a combination of Python's `yield` keyword, dictionaries (`book`, `books`), and list comprehensions to extract author names from a collection of book objects.

python
yield from {book.get("author") for book in books if book.get("author")}


**Breaking Down the Code**
-------------------------

*   **`yield from`**: This is an expression that deconstructs an iterable, like a list or another generator. It yields each element from the contained iterable, rather than simply copying the iterable and yielding from it.

*   `{...}`: This denotes a dictionary comprehension, which creates a new dictionary from an existing dictionary comprehension.

*   `book.get("author")`: For each book object (`book`), this selects the "value" associated with the key `"author"` in that book's dictionary representation. The `.get()` method is used instead of `.keys()`, `.values()`, or `.items()`, as it allows us to specify a default value if the key doesn't exist.

*   `for book in books`: This specifies our iterator over a collection of book objects (`books`).

*   `if book.get("author")`: We include only those books where there exists an `"author"` in the dictionary representation. The `.get()` method will return `None`, which is considered "falsy," so it wouldn't appear in our iteration.

**Example Use Case:**
--------------------

Let's create a few examples of books and use them to understand how this code works:

python
import json

class Book:
    def __init__(self, id, title, author):
        self.id = id
        self.title = title
        self.author = author

books = [
    Book(1, "To Kill a Mockingbird", "Harper Lee"),
    Book(2, "1984", None),
    Book(3, "Pride and Prejudice", "Jane Austen"),
]

# Convert our list of books into a dictionary representation
books_dict = [{key: value for key, value in book.__dict__.items() if not key.endswith('_')} for book in books]

yield from {book.get("author") for book in books_dict if book and 'author' in book}


**Output:**
---------

When you run the code above, it will output:

python
{'Harper Lee', None, 'Jane Austen'}


Each dictionary contained in `books_dict` has all its property names except the ones ending with `_`. The outer expression generates an iterator of only those who actually contain `"author"` as a key.